In [1]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from typing import Iterable, List
from model import Transformer
from data import fr_to_en
import utils
import torch.nn as nn
import pandas as pd
import json
import torch



### 토크나이징에 활용할 Vocab 생성

In [2]:
# 훈련 데이터 불러오기
# Fr -> En 번역을 위한 데이터셋(Multi-30k) 활용
fr_data = utils.open_text_set("data/training/train.fr")
eng_data = utils.open_text_set("data/training/train.en")

# Vocab 만들기 / 관련 함수는 utils.py 참조
try : 
  vocab_transform, token_transform = utils.make_vocab(fr_data, eng_data)
except :  
  # 오류 발생 시 spacy 설치 필요

  # spacy tokenizer 다운로드(en,fr)
  import spacy.cli
  spacy.cli.download("en_core_web_sm")
  spacy.cli.download("fr_core_news_sm")
  vocab_transform, token_transform = utils.make_vocab(fr_data, eng_data)

# param
SRC_LANGUAGE = "fr"
TGT_LANGUAGE = "en"

### `[tutorial] training`에서 학습한 모델 불러오기

In [3]:
with open('config/transformer.json', 'r') as file:
    param = json.load(file)
    print('Model_Parameters')
    print('-'*50)
    print(param)  

# multi-30k 데이터를 20번 epoch한 모델 불러오기
model = Transformer(**param)
model.load_state_dict(torch.load('model/model.pth'))

# 모델 평가모드로 변경
model.eval()

device = model.device

print('-'*50)
print(f'현재 devicde 설정값은 : "{model.device}" 입니다. 변경을 희망하실 경우 config/transformer.json을 수정해주세요.')
print('-'*50)

# loss_fn
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=1)


Model_Parameters
--------------------------------------------------
{'src_vocab_size': 11509, 'trg_vocab_size': 10837, 'src_pad_idx': 1, 'trg_pad_idx': 1, 'embed_size': 512, 'num_layers': 3, 'forward_expansion': 2, 'heads': 8, 'dropout': 0.1, 'device': 'cpu', 'max_length': 140}
--------------------------------------------------
현재 devicde 설정값은 : "cpu" 입니다. 변경을 희망하실 경우 config/transformer.json을 수정해주세요.
--------------------------------------------------


### 모델 테스트

* 아래 구현 된 test 함수를 통해 Transformer의 실제 문제 예측 과정을 이해할 수 있음.

* 모델은 한 번의 하나의 토큰을 생산하며 < bos > token을 시작으로 다음 토큰을 예측해 < eos > 토큰이 생성될때까지 반복 함.



In [5]:
# token을 단어로 바꾸기 위한 dict 생성, vocab의 key와 value 위치 변경
# 아래 helper 함수에서 활용됨.
decoder_en = {v:k for k,v in vocab_transform['en'].get_stoi().items()}
decoder_fr = {v:k for k,v in vocab_transform['fr'].get_stoi().items()}

def tokenizing_src(input_data:str) : 
    # input_data_tokenizing
    token_data = token_transform['fr'](input_data)
    vocab_src = vocab_transform['fr'](token_data)
    tokenized_src = [2] + vocab_src + [3]
    return tokenized_src

def select_random_item() :
    num = torch.randint(1,29000,(1,)).item()

    return fr_data[num], fr_data[num]

def test(model) :
    '''
    * validation은 문제와 정답이 모두 주어진다면 test는 문제만 제공하는 상황임.

    * test 함수를 통해 Transformer의 실제 문제 예측 과정을 이해할 수 있음.

    * Transformer는 문제와 정답이 있다면 답을 구하는 과정을 병렬적으로 수행할 수 있음.

    * 하지만 테스트에서는 정답이 주어지지 않으므로 한 번의 하나의 토큰을 생산함.

    * < bos > token을 시작으로 다음 토큰을 예상하며 < eos > 토큰이 생성될때까지 반복적으로 예측을 수행하게 되는 알고리즘이 필요함.

    * 아래의 test 함수를 다뤄보면서 Transformer의 데이터 처리 과정을 이해할 수 있음.

    '''

    model.eval()

    # 임의의 훈련 데이터 선별
    fr_item, en_item = select_random_item()
    
    print('입력 :', fr_item)

    # Input Data 토크나이징 
    tokenized_input = tokenizing_src(fr_item)
    max_length = int(len(tokenized_input) * 1.2)

    # src Tensor에 Token 저장
    src = torch.LongTensor(tokenized_input).unsqueeze(0).to(device)

    # trg Tensor 생성(1, max_length)
    trg = torch.zeros(1,max_length).type_as(src.data).to(device)

    # src encoding
    enc_src = model.encode(src)

    next_trg = 2 # 문장 시작 <bos> idx

    # 문장 예측 시작
    for i in range(0,max_length) :
        trg[0][i] = next_trg # token 저장

        logits = model.decode(src,trg,enc_src) # output 산출

        prd = logits.squeeze(0).max(dim=-1, keepdim=False)[1] # 예측 단어 중 max 추출
        next_word = prd.data[i] # i 번째 위치한 단어 추출
        next_trg = next_word.item() 
        if next_trg == 3 :
            # <eos> 나오면 종료
            trg[0][i] = next_trg
            break
    
    # <pad> 제거
    if 3 in trg[0] :
        eos_idx = int(torch.where(trg[0] == 3)[0][0])
        trg = trg[0][:eos_idx].unsqueeze(0)
    else :
        pass

    # 번역
    translation = [decoder_en[i] for i in trg.squeeze(0).tolist()]
    print('모델예측 :',' '.join(translation[1:]))
    

    print('정답 :', en_item)
    print('')
    print('주의! 29,000개의 제한된 데이터로 학습을 수행했으므로 완벽한 예측이 불가능함.')



test(model)


입력 : Un adolescent a un anneau argenté saillant de son nez.
모델예측 : A little boy is holding a piece of a large white shirt is holding
정답 : A teenage boy has a silver ring protruding from his nose.

주의! 29,000개의 제한된 데이터로 학습을 수행했으므로 완벽한 예측이 불가능함.
